In [ ]:
import numpy as np
import scipy as sp
import scipy.optimize
from dataclasses import dataclass
from matplotlib import pyplot as plt

def dot(a, b, **kwargs):
    return np.multiply(a, b).sum(axis=-2, **kwargs)

def make_vec3(x, y, z):
    return np.array((x, y, z)).reshape(3, 1)

@dataclass
class SG:
    axis: np.ndarray
    amplitude: np.ndarray
    sharpness: np.ndarray

    def eval(self, v):
        r = self.amplitude * np.exp(self.sharpness * (dot(v, self.axis, keepdims=True) - 1))
        s = r.sum(axis=(0, 1))
        return s

    def jac(self, v):
        f = self.amplitude * np.exp(self.sharpness * (dot(v, self.axis, keepdims=True) - 1))
        d_axis = f * self.sharpness * v
        d_amp = f / self.amplitude
        d_sh = f * (dot(v, self.axis, keepdims=True) - 1)
        j = np.concatenate((d_axis, d_amp, d_sh), axis=1)
        return j

def polar_to_r3(phi, axis=0):
    phi = phi if len(np.shape(phi)) > 0 else np.reshape(phi, 1)
    p = np.stack((np.cos(phi), np.zeros(np.shape(phi)), np.sin(phi)), axis=axis)
    return p

def polar_to_r3_diff(phi, axis=0):
    phi = phi if len(np.shape(phi)) > 0 else np.reshape(phi, 1)
    p = np.stack((-np.sin(phi), np.zeros(np.shape(phi)), np.cos(phi)), axis=axis)
    return p

def spherical_to_r3(phi, theta, axis=0):
    z = np.cos(theta)
    r = np.sin(theta)
    return np.stack((np.cos(phi) * r, np.sin(phi) * r, z), axis=axis)

def spherical_to_r3_diff(phi, theta, axis=0):
    z = np.cos(theta)
    r = np.sin(theta)
    dphi = np.stack((-np.sin(phi) * r, np.cos(phi) * r, np.zeros(z.shape)), axis=axis)
    dtheta = np.stack((np.cos(phi) * z, np.sin(phi) * z, -r), axis=axis)
    return (dphi, dtheta)

def newton(f, df, x0, n):
    xp = x0
    x = x0
    for i in range(n):
        x = xp - f(xp) / df(xp)
        xp = x
    return x

def F_schlick(f0, NoL):
    return f0 + (1 - f0) * (1 - NoL) ** 5

def G_smith(alpha, NoV, NoL):
    alpha2 = alpha ** 2;
    NoV2 = NoV ** 2
    NoL2 = NoL ** 2
    A_V = np.sqrt(1 + alpha2 * (1 - NoV2) / NoV2)
    A_L = np.sqrt(1 + alpha2 * (1 - NoL2) / NoL2)
    return 2 / (A_V + A_L)

def D_ggx(alpha, NoH):
    alpha2 = alpha ** 2
    return alpha2 / (np.pi * (1 + NoH ** 2 * (alpha2 - 1)) ** 2)

def BRDF(f0, alpha, N, V, L, mul_NoL=False):
    H = L + V
    H = H / np.linalg.norm(H, axis=0)
    HoV = dot(V, H)
    NoV = dot(N, V)
    NoL = dot(N, L)
    NoH = dot(N, H)
    F = F_schlick(f0, HoV)
    G = G_smith(alpha, NoV, NoL)
    D = D_ggx(alpha, NoH)
    if mul_NoL:
        return np.where(NoL > 0, F * G * D / (4 * NoV), 0)
    return np.where(NoL > 0, F * G * D / (4 * NoV * NoL), 0)

def r2_seq(n, seed=0.5):
    g = 1.32471795724474602596
    a1 = 1 / g
    a2 = a1 ** 2
    x, _ = np.modf(seed + a1 * n) 
    y, _ = np.modf(seed + a2 * n)
    return np.stack((x, y))

def uniform_sample_hemisphere(Xi):
    phi = 2 * np.pi * Xi[0]
    z = Xi[1]
    theta = np.acos(z)
    return (phi, theta)

def uniform_sample_sphere(Xi):
    phi = 2 * np.pi * Xi[0]
    z = 2 * Xi[1] - 1
    theta = np.acos(z)
    return (phi, theta)
        

In [ ]:
# Fit visibility SG

@dataclass
class Cone:
    axis: np.ndarray
    aperture: np.ndarray

    def eval(self, v):
        return np.where(dot(v, self.axis) > np.cos(self.aperture), 1.0, 0.0)

def ka_from_lambda(l):
    return 2.0 * (l + np.exp(-l) - 1.0) / (l * l)

def ka_from_lambda_diff(l):
    return -2.0 * (l - 2.0 + np.exp(-l) * (l + 2.0)) / (l * l * l)

def l0_from_ka(ka):
    return 2.0 / (ka + 0.01) * (1.0 - ka) + 3.0 * (1.0 - ka) * ka

for ka in (0.1, 0.5, 0.9):
    N = make_vec3(0, 0, 1)
    
    l0 = l0_from_ka(ka)
    l1 = newton(lambda l: ka_from_lambda(l) - ka, ka_from_lambda_diff, l0, 1)
    sg = SG(np.reshape(N, (1, 3, 1)), np.reshape(1, (1, 1, 1)), np.reshape(l1, (1, 1, 1)))

    aperture = np.acos(np.sqrt(1.0 - ka))
    c = Cone(N, aperture)

    phi = np.linspace(0, np.pi, num=100)
    axis = polar_to_r3(phi)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

    plt.title(f"Visibility fit for ka={ka}")
    ax.plot(phi, c.eval(axis), label=f"Cone")
    ax.plot(phi, sg.eval(axis), label=f"SG")
    ax.plot(phi, c.eval(axis) * np.sin(phi), label = f"Cosine-weighted cone")
    ax.plot(phi, sg.eval(axis) * np.sin(phi), label = f"Cosine-weighted SG")
    ax.set_rmax(1)
    ax.set_rticks([0.0, 0.5, 1])
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"Visibility fit for ka={ka}")
    ax.plot(phi - np.pi / 2.0, c.eval(axis), label=f"Cone")
    ax.plot(phi - np.pi / 2.0, sg.eval(axis), label=f"SG")
    ax.plot(phi - np.pi / 2.0, c.eval(axis) * np.sin(phi), label = f"Cosine-weighted cone")
    ax.plot(phi - np.pi / 2.0, sg.eval(axis) * np.sin(phi), label = f"Cosine-weighted SG")
    ax.set_ylim(1)
    ax.set_yticks([0.0, 0.5, 1])
    ax.grid(True)
    ax.legend()
    plt.show()

In [ ]:
# Fit normal distribution SG

for roughness in (0.1, 0.5, 0.9):
    N = make_vec3(0, 0, 1)
    
    alpha = roughness ** 2
    alpha2 = alpha ** 2

    phi = np.linspace(0, np.pi, num=100)
    H = polar_to_r3(phi)
    d = D_ggx(alpha, dot(N, H))

    def eval_ggx_sg(phi, *args):
        args = np.array(args).reshape(-1, 2, 1)
        
        amp = args[:, 0]
        sh = args[:, 1]
        axis = np.full(amp.shape, np.pi / 2)
        
        return SG(polar_to_r3(axis, axis=1), np.expand_dims(amp, -1), np.expand_dims(sh, -1)).eval(polar_to_r3(phi))

    def jac_ggx_sg(phi, *args):
        v = polar_to_r3(phi)
        args = np.array(args).reshape(-1, 2, 1)
        
        amp = args[:, 0]
        sh = args[:, 1]
        u = polar_to_r3(np.full(amp.shape, np.pi / 2), axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        
        j = sg.jac(v)
        j = np.vstack(j[:,3:]).transpose()

        return j

    d_sgs = []

    for i in range(2):
        k = i + 1
        try:
            p0 = np.tile((1 / (np.pi * alpha2 * k), 2 / alpha2), k)
            p0, _ = sp.optimize.curve_fit(eval_ggx_sg, phi, d, p0=p0, jac=jac_ggx_sg)
            scale = 1 / np.abs(p0)
            lb = np.tile((-np.inf, 0.0), k)
            ub = np.tile((np.inf, np.inf), k)
            p0 = np.clip(p0, lb, ub)
            popt, pcov = sp.optimize.curve_fit(eval_ggx_sg, phi, d, p0=p0, bounds=(lb, ub), x_scale=scale, jac=jac_ggx_sg)
            print(f"Fit {k} SG(s):")
            print("Optimal parameters:\n{}".format(popt.reshape(-1, 2)))
            print("CN: {}".format(np.linalg.cond(pcov)))
            sigma_2 = np.diag(pcov).reshape(-1, 2)
            sigma = np.sqrt(sigma_2)
            eps = np.abs(sigma / popt.reshape(-1, 2))
            print(f"Sigma^2:\n{sigma_2}")
            print(f"Sigma:\n{sigma}")
            print(f"Epsilon:\n{eps}")
            print("")
            d_sgs.append(popt)
        except RuntimeError:
            break

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    plt.title(f"D fit for Roughness={roughness}")
    ax.plot(phi, d, label="GGX")
    for i, d_sg in enumerate(d_sgs):
        k = i + 1
        ax.plot(phi, eval_ggx_sg(phi, d_sg), label=f"{k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"D fit for Roughness={roughness}")
    ax.plot(phi - np.pi / 2.0, d, label="GGX")
    for i, d_sg in enumerate(d_sgs):
        k = i + 1
        ax.plot(phi - np.pi / 2.0, eval_ggx_sg(phi, d_sg), label=f"{k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

In [ ]:
# Fit BRDF SG

for roughness in (0.1, 0.5, 0.9):
    f0 = 0.04
    alpha = roughness ** 2
    alpha2 = alpha ** 2

    N_phi = np.pi / 2
    V_phi = 3 * np.pi / 4
    R_phi = 2 * N_phi - V_phi
    N = polar_to_r3(N_phi)
    V = polar_to_r3(V_phi)

    phi = np.linspace(0, np.pi, num=100)
    L = polar_to_r3(phi)
    f = BRDF(f0, alpha, N, V, L)

    def eval_brdf_sg(phi, *args):
        args = np.array(args).reshape(-1, 3, 1)
        axis = args[:, 0]
        amp = args[:, 1]
        sh = args[:, 2]
        return SG(polar_to_r3(axis, axis=1), np.expand_dims(amp, -1), np.expand_dims(sh, -1)).eval(polar_to_r3(phi))
  
    def jac_brdf_sg(phi, *args):
        v = polar_to_r3(phi)
        args = np.array(args).reshape(-1, 3, 1)

        axis = args[:, 0]
        amp = args[:, 1]
        sh = args[:, 2]
        u = polar_to_r3(axis, axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        
        du = polar_to_r3_diff(axis, axis=1)
        
        j = sg.jac(v)
        du = dot(du, j[:, :3], keepdims=True)
        j = np.concatenate((du, j[:, 3:]), axis=1)
        j = np.vstack(j).transpose()

        return j

    brdf_sgs = []

    MIN_SG = 2
    MAX_SG = 4

    for k in range(MIN_SG, MAX_SG + 1):
        try:
            p0 = np.tile((R_phi, 1 / (np.pi * alpha2 * k), 2 / alpha2), k)
            popt, pcov = sp.optimize.curve_fit(eval_brdf_sg, phi, f, p0=p0, jac=jac_brdf_sg, maxfev=10_000)
            print(f"Fit {k} SG(s):")
            print("Optimal parameters:\n{}".format(popt.reshape(-1, 3)))
            print("CN: {}".format(np.linalg.cond(pcov)))
            sigma_2 = np.diag(pcov).reshape(-1, 3)
            sigma = np.sqrt(sigma_2)
            eps = np.abs(sigma / popt.reshape(-1, 3))
            print(f"Sigma^2:\n{sigma_2}")
            print(f"Sigma:\n{sigma}")
            print(f"Epsilon:\n{eps}")
            print("")
            brdf_sgs.append(popt)
        except RuntimeError:
            break

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    plt.title(f"BRDF fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi, f, label="f")
    for k, brdf_sg in enumerate(brdf_sgs, start=MIN_SG):
        ax.plot(phi, eval_brdf_sg(phi, brdf_sg), label=f"f: {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"BRDF fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi - np.pi / 2, f, label="f")
    for k, brdf_sg in enumerate(brdf_sgs, start=MIN_SG):
        ax.plot(phi - np.pi / 2, eval_brdf_sg(phi, brdf_sg), label=f"f: {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

In [ ]:
# Fit BRDF (N, L) SG

for roughness in (0.1, 0.5, 0.9):
    f0 = 0.04
    alpha = roughness ** 2
    alpha2 = alpha ** 2

    N_phi = np.pi / 2
    V_phi = 3 * np.pi / 4
    R_phi = 2 * N_phi - V_phi
    N = polar_to_r3(N_phi)
    V = polar_to_r3(V_phi)

    phi = np.linspace(-np.pi / 2, 3 * np.pi / 2, num=100)
    L = polar_to_r3(phi)
    f_NoL = BRDF(f0, alpha, N, V, L, mul_NoL=True)

    def eval_brdf_sg(phi, *args):
        args = np.array(args).reshape(-1, 3, 1)
        axis = args[:, 0]
        amp = args[:, 1]
        sh = args[:, 2]  
        return SG(polar_to_r3(axis, axis=1), np.expand_dims(amp, -1), np.expand_dims(sh, -1)).eval(polar_to_r3(phi))

    def jac_brdf_sg(phi, *args):
        v = polar_to_r3(phi)
        args = np.array(args).reshape(-1, 3, 1)

        axis = args[:, 0]
        amp = args[:, 1]
        sh = args[:, 2]
        u = polar_to_r3(axis, axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        
        du = polar_to_r3_diff(axis, axis=1)
        
        j = sg.jac(v)
        du = dot(du, j[:, :3], keepdims=True)
        j = np.concatenate((du, j[:, 3:]), axis=1)
        j = np.vstack(j).transpose()

        return j

    brdf_NoL_sgs = []

    MIN_SG = 2
    MAX_SG = 4

    for k in range(MIN_SG, MAX_SG + 1):
        try:
            p0 = np.tile((R_phi, 1 / (np.pi * alpha2 * k), 2 / alpha2), k)
            popt, pcov = sp.optimize.curve_fit(eval_brdf_sg, phi, f_NoL, p0=p0, jac=jac_brdf_sg, maxfev=10_000)
            print(f"Fit {k} SG(s):")
            print("Optimal parameters:\n{}".format(popt.reshape(-1, 3)))
            # print("CN: {}".format(np.linalg.cond(pcov)))
            sigma_2 = np.diag(pcov).reshape(-1, 3)
            sigma = np.sqrt(sigma_2)
            eps = np.abs(sigma / popt.reshape(-1, 3))
            print(f"Sigma^2:\n{sigma_2}")
            print(f"Sigma:\n{sigma}")
            print(f"Epsilon:\n{eps}")
            print("")
            brdf_NoL_sgs.append(popt)
        except RuntimeError:
            break

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    plt.title(f"BRDF * (N, L) fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi, f_NoL, label="f * (N, L)")
    for k, brdf_NoL_sg in enumerate(brdf_NoL_sgs, start=MIN_SG):
        ax.plot(phi, eval_brdf_sg(phi, brdf_NoL_sg), label=f"f * (N, L): {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"BRDF * (N, L) fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi - np.pi / 2, f_NoL, label="f * (N, L)")
    for k, brdf_NoL_sg in enumerate(brdf_NoL_sgs, start=MIN_SG):
        ax.plot(phi - np.pi / 2, eval_brdf_sg(phi, brdf_NoL_sg), label=f"f * (N, L): {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

In [ ]:
# Fit BRDF SG in 3D

for roughness in (0.1, 0.5, 0.9):
    f0 = 0.04
    alpha = roughness ** 2
    alpha2 = alpha ** 2

    N_phi = np.pi / 2
    V_phi = 3 * np.pi / 4
    R_phi = 2 * N_phi - V_phi
    N = polar_to_r3(N_phi)
    V = polar_to_r3(V_phi)
    R = polar_to_r3(R_phi)

    Xi = r2_seq(np.arange(10000))
    (phi, theta) = uniform_sample_hemisphere(Xi)
    L = spherical_to_r3(phi, theta)
    f = BRDF(f0, alpha, N, V, L)
    
    def eval_brdf_sg(X, *args):
        L = spherical_to_r3(*X) if len(X.shape) == 2 else polar_to_r3(X)
        args = np.array(args).reshape(-1, 4, 1)
        phi = args[:, 0]
        theta = args[:, 1]
        amp = args[:, 2]
        sh = args[:, 3]
        u = spherical_to_r3(phi, theta, axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        return sg.eval(L)
  
    def jac_brdf_sg(X, *args):
        L = spherical_to_r3(*X) if len(X.shape) == 2 else polar_to_r3(X)
        args = np.array(args).reshape(-1, 4, 1)

        phi = args[:, 0]
        theta = args[:, 1]
        amp = args[:, 2]
        sh = args[:, 3]
        u = spherical_to_r3(phi, theta, axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        
        du_dphi, du_dtheta = spherical_to_r3_diff(phi, theta, axis=1)
        
        j = sg.jac(L)
        du_dphi = dot(du_dphi, j[:, :3], keepdims=True)
        du_dtheta = dot(du_dtheta, j[:, :3], keepdims=True)
        j = np.concatenate((du_dphi, du_dtheta, j[:, 3:]), axis=1)
        j = np.vstack(j).transpose()

        return j

    brdf_sgs = []

    MIN_SG = 2
    MAX_SG = 5

    for k in range(MIN_SG, MAX_SG + 1):
        try:
            p0 = np.tile((np.arctan2(R[1], R[0])[0], np.acos(R[2])[0], 1 / (np.pi * alpha2 * k), 2 / alpha2), k)
            popt, pcov = sp.optimize.curve_fit(eval_brdf_sg, (phi, theta), f, p0=p0, jac=jac_brdf_sg, maxfev=10_000)
            print(f"Fit {k} SG(s):")
            print("Optimal parameters:\n{}".format(popt.reshape(-1, 4)))
            # print("CN: {}".format(np.linalg.cond(pcov)))
            sigma_2 = np.diag(pcov).reshape(-1, 4)
            sigma = np.sqrt(sigma_2)
            eps = np.abs(sigma / popt.reshape(-1, 4))
            print(f"Sigma^2:\n{sigma_2}")
            print(f"Sigma:\n{sigma}")
            print(f"Epsilon:\n{eps}")
            print("")
            brdf_sgs.append(popt)
        except RuntimeError:
            break

    phi = np.linspace(0, np.pi, num=100)
    L = polar_to_r3(phi)
    f = BRDF(f0, alpha, N, V, L)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    plt.title(f"BRDF fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi, f, label="f")
    for k, brdf_sg in enumerate(brdf_sgs, start=MIN_SG):
        ax.plot(phi, eval_brdf_sg(phi, brdf_sg), label=f"f: {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"BRDF fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi - np.pi / 2, f, label="f")
    for k, brdf_sg in enumerate(brdf_sgs, start=MIN_SG):
        ax.plot(phi - np.pi / 2, eval_brdf_sg(phi, brdf_sg), label=f"f: {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    W = 30
    H = 30
    
    Xix = np.linspace(0, 1, num=W)
    Xiy = np.linspace(0, 1, num=H)
    Xix, Xiy = np.meshgrid(Xix, Xiy)
    Xix = Xix.flatten()
    Xiy = Xiy.flatten()
    X = np.stack(uniform_sample_hemisphere((Xix, Xiy)))
    L = spherical_to_r3(*X)
    f = BRDF(f0, alpha, N, V, L).reshape(H, W)
    L = L.reshape(3, H, W)
    P = L * f

    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    plt.title(f"BRDF fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot_surface(P[0], P[1], P[2], antialiased=False, label="f")
    for k, sg in enumerate(brdf_sgs, start=MIN_SG):
        f = eval_brdf_sg(X, sg).reshape(H, W)
        P = L * f
        try:
            ax.plot_surface(P[0], P[1], P[2], antialiased=False, label=f"f: {k} SG(s)")
        except ValueError:
            continue
    ax.legend()
    plt.show()

In [ ]:
# Fit BRDF (N, L) SG in 3D

for roughness in (0.1, 0.5, 0.9):
    f0 = 0.04
    alpha = roughness ** 2
    alpha2 = alpha ** 2

    N_phi = np.pi / 2
    V_phi = 3 * np.pi / 4
    R_phi = 2 * N_phi - V_phi
    N = polar_to_r3(N_phi)
    V = polar_to_r3(V_phi)

    Xi = r2_seq(np.arange(10000))
    (phi, theta) = uniform_sample_sphere(Xi)
    L = spherical_to_r3(phi, theta)
    f = BRDF(f0, alpha, N, V, L, mul_NoL=True)
    
    def eval_brdf_sg(X, *args):
        L = spherical_to_r3(*X) if len(X.shape) == 2 else polar_to_r3(X)
        args = np.array(args).reshape(-1, 4, 1)
        phi = args[:, 0]
        theta = args[:, 1]
        amp = args[:, 2]
        sh = args[:, 3]
        u = spherical_to_r3(phi, theta, axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        return sg.eval(L)
  
    def jac_brdf_sg(X, *args):
        L = spherical_to_r3(*X) if len(X.shape) == 2 else polar_to_r3(X)
        args = np.array(args).reshape(-1, 4, 1)

        phi = args[:, 0]
        theta = args[:, 1]
        amp = args[:, 2]
        sh = args[:, 3]
        u = spherical_to_r3(phi, theta, axis=1)
        sg = SG(u, np.expand_dims(amp, -1), np.expand_dims(sh, -1))
        
        du_dphi, du_dtheta = spherical_to_r3_diff(phi, theta, axis=1)
        
        j = sg.jac(L)
        du_dphi = dot(du_dphi, j[:, :3], keepdims=True)
        du_dtheta = dot(du_dtheta, j[:, :3], keepdims=True)
        j = np.concatenate((du_dphi, du_dtheta, j[:, 3:]), axis=1)
        j = np.vstack(j).transpose()

        return j

    brdf_sgs = []

    MIN_SG = 2
    MAX_SG = 5

    for k in range(MIN_SG, MAX_SG + 1):
        try:
            p0 = np.tile((np.arctan2(R[1], R[0])[0], np.acos(R[2])[0], 1 / (np.pi * alpha2 * k), 2 / alpha2), k)
            popt, pcov = sp.optimize.curve_fit(eval_brdf_sg, (phi, theta), f, p0=p0, jac=jac_brdf_sg, maxfev=10_000)
            print(f"Fit {k} SG(s):")
            print("Optimal parameters:\n{}".format(popt.reshape(-1, 4)))
            # print("CN: {}".format(np.linalg.cond(pcov)))
            sigma_2 = np.diag(pcov).reshape(-1, 4)
            sigma = np.sqrt(sigma_2)
            eps = np.abs(sigma / popt.reshape(-1, 4))
            print(f"Sigma^2:\n{sigma_2}")
            print(f"Sigma:\n{sigma}")
            print(f"Epsilon:\n{eps}")
            print("")
            brdf_sgs.append(popt)
        except RuntimeError:
            break

    phi = np.linspace(-np.pi / 2, 3 * np.pi / 2, num=100)
    L = polar_to_r3(phi)
    f = BRDF(f0, alpha, N, V, L, mul_NoL=True)

    fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
    plt.title(f"BRDF (N, L) fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi, f, label="f")
    for k, brdf_sg in enumerate(brdf_sgs, start=MIN_SG):
        ax.plot(phi, eval_brdf_sg(phi, brdf_sg), label=f"f: {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    fig, ax = plt.subplots()
    plt.title(f"BRDF (N, L) fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot(phi - np.pi / 2, f, label="f")
    for k, brdf_sg in enumerate(brdf_sgs, start=MIN_SG):
        ax.plot(phi - np.pi / 2, eval_brdf_sg(phi, brdf_sg), label=f"f: {k} SG(s)")
    ax.grid(True)
    ax.legend()
    plt.show()

    W = 30
    H = 30
    
    Xix = np.linspace(0, 1, num=W)
    Xiy = np.linspace(0, 1, num=H)
    Xix, Xiy = np.meshgrid(Xix, Xiy)
    Xix = Xix.flatten()
    Xiy = Xiy.flatten()
    X = np.stack(uniform_sample_hemisphere((Xix, Xiy)))
    L = spherical_to_r3(*X)
    f = BRDF(f0, alpha, N, V, L, mul_NoL=True).reshape(H, W)
    L = L.reshape(3, H, W)
    P = L * f

    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    plt.title(f"BRDF (N, L) fit for Roughness={roughness}, f0={f0}, V_phi={V_phi}")
    ax.plot_surface(P[0], P[1], P[2], antialiased=False, label="f")
    for k, sg in enumerate(brdf_sgs, start=MIN_SG):
        f = eval_brdf_sg(X, sg).reshape(H, W)
        P = L * f
        try:
            ax.plot_surface(P[0], P[1], P[2], antialiased=False, label=f"f: {k} SG(s)")
        except ValueError:
            continue
    ax.legend()
    plt.show()